# Outlook Email Automation with Python
---
**Business Problem:**

According to a study by [Mckinsey](https://www.mckinsey.com/industries/technology-media-and-telecommunications/our-insights/the-social-economy), employees spend roughly 28% of their workweek managing emails. Many of us waste time on repetitive email tasks such as sending multiple emails, attaching files to emails, and saving attachments from emails.

**Proposed Solution:**

We can use Python to automate several email-related tasks in order to reallocate some of the time spent on email management to more value-adding activities.

**Expected Results:**

1. Increased workplace productivity and efficiency - Employees will have more time to spend on more productive activities.
2. Reduced errors - There will be fewer errors and rework since there is less manual intervention.

## Getting started

This project demonstrates some of the email operations that we can automate with Python.

Let's start by installing **pywin32** via pip. 

In [ ]:
pip install pywin32

In order to use pywin32, we need to import **win32.client**. We also have to import other modules that we'll use for this project.

In [2]:
import win32com.client as client
import pathlib
from time import sleep
import pandas as pd
import os

Next, we have to connect to Outlook.

In [3]:
outlook = client.Dispatch('Outlook.Application')

## Sending emails

In [4]:
email_to = 'recipient@email.com' 

### Sending a plain text email

To send a plain text email:
1. Instantiate a new email using **outlook.CreateItem(0)**. 
2. Add a recipient by assigning the email address to **msg.To**, **msg.CC** or **msg.BCC**.
3. Assign the subject and the plain text message to **msg.Subject** and **msg.Body**, respectively.
4. Save the email in Drafts using **msg.Save()** or send it directly using **msg.Send()**.

In [ ]:
msg = outlook.CreateItem(0)  # new email

msg.To = email_to
msg.Subject = 'Test'
msg.Body = 'Hello! This is a test email'  # plain text message

msg.Save()
msg.Send()

### Sending an HTML-formatted email

To send an email in HTML format, simply repeat the steps for creating a plain text email; but instead of setting a plain text message to msg.Body, assign an HTML-formatted message to **msg.HTMLBody**.

In [ ]:
msg = outlook.CreateItem(0)

msg.To = email_to
msg.Subject = 'Test'
body = '''
<h1 style="font-family: Tahoma; font-size: 36; font-weight: bold; color: #00c0ef;">
    Hello! 
</h1>
<span style="font-family: Arial; font-size: 28; color: #0073b7;">
    This is a test email.
</span>
<br>
<img src="https://image.cnbcfm.com/api/v1/image/102962809-maxresdefault.jpg?" width=30%>
'''
msg.HTMLBody = body  # html message

msg.Send()

### Sending an email with attachment

To send an email with attachment, follow the procedure for creating an email (plain or HTML) then use **msg.Attachments.Add(file)** to attach a file. Note that the absolute path of the file should be used when adding attachments to Outlook.

In [32]:
msg = outlook.CreateItem(0)

msg.To = email_to
msg.Subject = 'Test'

test_file = str(pathlib.Path('sample_files/test_file.pdf').absolute())  # absolute path
msg.Attachments.Add(test_file)  # attach file

body = '''
<h1 style="font-family: Tahoma; font-size: 36; font-weight: bold; color: #00c0ef;">
    Hello! 
</h1>
<span style="font-family: Arial; font-size: 28; color: #0073b7;">
    This is a test email.
</span>
'''
msg.HTMLBody = body

msg.Send()

### Sending an email to multiple recipients
Note: Outlook has a cap of 500 recipients for each individual email.

**Email with less than 500 recipients**

To send an email to less than 500 recipients, simply join the email addresses using a semi-colon and assign it to **msg.To**.

In [ ]:
# For demonstration purposes, a distribution list with less than 500 recipients was created.
dlist = [email_to for _ in range(450)] 

msg = outlook.CreateItem(0)

msg.To = ';'.join(dlist)  # join the email addresses
msg.Subject = 'Test'
msg.Body = 'Hello! This is a test email' 

msg.Send()

**Email with more than 500 recipients.**

To send an email to more than 500 recipients:
1. Generate chunks of the email address list, which contains at most 500 email addresses each.
2. Loop over the chunks and create a new email for each chunk.
3. Join the email addresses using a semi-colon and assign it to **msg.To**

In [ ]:
# For demonstration purposes, a distribution list with more than 500 recipients was created.
dlist = [email_to for _ in range(650)]

chunks = [dlist[i:i+500] for i in range(0,len(dlist),500)]  # chunks with <= 500 addresses each

for chunk in chunks:
    msg = outlook.CreateItem(0)  # new email for each chunk
    msg.To = ';'.join(chunk)
    msg.Subject = 'Test'
    msg.Body = 'Hello! This is a test email'
    msg.Send()  

### Sending multiple emails
Note: Outlook has a cap of 30 outgoing emails per minute.

**Emails sent individually to less than 30 email addresses**

To send separate emails to less than 30 recipients, loop over the list of recipients and create/send a new email for each. By doing so, the emails are sent one after another.

In [ ]:
# For demonstration purposes, a distribution list in csv format with <30 names and email addresses in the first and second columns is created.
emp = pd.read_csv('sample_files/employees_1.csv') 

dlist = emp.values.tolist()

for name, email in dlist:
    msg = outlook.CreateItem(0)  # new email for each recipient
    msg.To = email
    msg.Subject = 'Test'
    msg.Body = f'Hello, {name}! This is a test email.'
    msg.Send()

**Emails sent individually to more than 30 email addresses**

To send separate emails to more than 30 recipients:
1. Generate chunks of the recipients list, which contains at most 30 email addresses each.
2. Loop over the chunks and create/send a new email for each recipient in the chunk.
3. Add a 60 second sleep before proceeding to the next chunk because the cap only allows 30 outgoing messages per minute.

In [ ]:
# For demonstration purposes, a distribution list in csv format with >30 names and email addresses in the first and second columns is created.
emp = pd.read_csv('sample_files/employees_2.csv')
dlist = emp.values.tolist() 

chunks = [dlist[i:i+30] for i in range(0,len(dlist),30)]  # chunks with <= 30 recipients each

for chunk in chunks:
    for name, email in chunk:
        msg = outlook.CreateItem(0)
        msg.To = email
        msg.Subject = 'Test'
        msg.Body = f'Hello, {name}! This is a test email.'
        msg.Send()
    sleep(60)  # wait 1 minute before processing the next chunk

### Sending emails with embedded spreadsheet (Excel) table

To add a table into the body of an email, 
1. Read the table from the Excel file into a pandas DataFrame.
2. Convert the dataframe to HTML format.
3. Place the HTML-formatted table in the HTML body.

In [12]:
tbl = pd.read_excel('sample_files/employees.xlsx')  # table as dataframe
html_tbl = tbl.to_html(index=False)  # table in html format

msg = outlook.CreateItem(0)

msg.To = email_to
msg.Subject = 'Test'
msg.HTMLBody = f'''
<span style="font-family: 'Arial'; font-size: 15; color: #0000ff;">
This is a test email with a table.
</span>
<br><br>
{html_tbl}
'''

msg.Send()

## Organizing Outlook folders and emails

### Folder management

In order to access default folders in Outlook, we need to get the NameSpace object first. Go to this [page](https://learn.microsoft.com/en-us/office/vba/api/outlook.namespace) to learn more about Outlook's NameSpace object.

In [5]:
namespace = outlook.GetNamespace("MAPI")  # get namespace object

Once the NameSpace object is returned, we can now access the default folders of Outlook such as Inbox and Drafts. Refer to this [link](https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders) for more information on Outlook's default folders.

In [6]:
# some of Outlook's default folders
inbox = namespace.GetDefaultFolder(6)
drafts = namespace.GetDefaultFolder(16)

In [7]:
# all folders in my Inbox
for folder in inbox.Folders:
    print(folder.Name)

Important
Promotions


Folders inside the default folders can be accessed in three different ways. Below shows how to get the first folder in my Inbox which is *Important* 

In [8]:
important = inbox.Folders['Important']
important = inbox.Folders[0]  # zero-indexed
important = inbox.Folders.Item(1)

Below shows how to add, move and delete folders. Check this [link](https://docs.microsoft.com/en-us/office/vba/api/outlook.folder) for other folder methods and properties.

In [9]:
social = inbox.Folders.Add('Social')
social.MoveTo(important)
social.Delete()

### Email management

Emails inside the default folders can be accessed in two ways. Below shows how to get the first email in my Inbox.

In [14]:
message = inbox.Items[0]  # zero-indexed
message = inbox.Items.Item(1)

In [13]:
# all emails in my Inbox
for item in inbox.Items:
    print(item.Subject)

Edit your presentation notes in PowerPoint Presenter view
Easily manipulate text and arrays with new functions in Excel
Dev Essentials - Build and test low-code apps and workflows for free
Updates to our terms of use
Dev Essentials - Announcing: .NET Multi-platform App UI is generally available
Preview the new Outlook for Windows
Make the most of Excel on your iPhone or iPad


Below demonstrates how to filter and move emails. Refer to this [link](https://docs.microsoft.com/en-us/office/vba/api/Outlook.MailItem) for other mail methods and properties.

In [15]:
# filter based on subject
for item in inbox.Items:
    if 'dev' in item.Subject.lower():
        item.Move(important)

In [16]:
# emails with attachment
for item in inbox.Items:
    if item.Attachments.Count > 0:
        item.Move(important)

## Reading emails

### Getting email details

To store email details in a pandas DataFrame:
1. Identify the folder where the desired emails are saved.
2. Access the emails and get the email information needed. Store the information in a list.
3. Convert the list into a DataFrame.

In [17]:
# For demonstation purposes, only the date sent, sender's email address and email subject are used.
inbox = namespace.GetDefaultFolder(6)  # source folder

email_list = [(str(item.SentOn).split('+')[0], item.SenderEmailAddress, item.Subject) for item in inbox.Items]

df = pd.DataFrame(email_list, columns=['Date Sent', 'Sender', 'Subject'])
print(df)

             Date Sent                                 Sender  \
0  2022-05-20 02:12:45             reply@e-mail.microsoft.com   
1  2022-04-27 02:10:34             reply@e-mail.microsoft.com   
2  2022-06-29 04:41:43        msa@communication.microsoft.com   
3  2022-06-24 02:15:16             reply@e-mail.microsoft.com   
4  2022-06-06 02:01:06  MicrosoftOfficeApps@email2.office.com   

                                             Subject  
0  Edit your presentation notes in PowerPoint Pre...  
1  Easily manipulate text and arrays with new fun...  
2                        Updates to our terms of use  
3                Preview the new Outlook for Windows  
4      Make the most of Excel on your iPhone or iPad  


### Saving email attachments

To save files attached in an email:
1. Identify the directory where you want the attachments saved.
2. Access the desired emails and save the attachments in the directory.

In [18]:
save_dir = 'C:\\Users\\NicoleSumalnap\\{}'

for item in inbox.Items:
    if item.Attachments.Count > 0:
        for file in item.Attachments:  # in case there are multiple attachments in the email
            file.SaveAsFile(save_dir.format(file.FileName))  # you can change the filename here